# TO DO
1. extract all text untill next example
2. clean the collected urls to only include lates version ( removes duplication)
https://ppubs.uspto.gov/pubwebapp/static/pages/ppubsbasic.html

# Imports

In [1]:
from collection import collect_urls,download_files,unzip_files
from utils import extract_examples_start_w_word,find_doc_number,process_siblings,extract_num_dot_examples,extract_examples_w_word,extract_experiments_w_heading,save_as_json,remove_duplicate_docs

# Data Collection

In [305]:
start_date = input("Enter start date YYYY-MM: ")
end_date = input("Enter end date YYYY-MM: ")
download_path = "patents_data"
unzip_path = "unzipped_patents_data"

files = collect_urls(start_date, end_date)
if files:
    total_size = sum(file["fileSize"] / 1024 / 1024 for file in files)
    print(
        f"\nTotal file size: {round(total_size / 1024, 2)} GB between {start_date} and {end_date}\n"
    )
    download_files(files, download_path)
    unzip_files(download_path, unzip_path)

Urls collected for the period from 2023-01 to 2023-05 successfully


# Processing

In [2]:
with open("unzipped_patents_data\\ipa230105.xml", "r", encoding="utf-8") as file:
    content = file.read()
xml_parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')

In [3]:
xml_no_dup = remove_duplicate_docs(xml_parts)

In [3]:
# from bs4 import BeautifulSoup

# def extract_experiments_w_heading(text):
#     """Extracts the 'Examples' section and its experiments from a patent text."""

#     # Use BeautifulSoup to parse the structure (for HTML-like patents)
#     soup = BeautifulSoup(text, "html.parser")

#     # Find the "EXAMPLES" section heading
#     examples_heading = soup.find(
#         lambda tag: tag.name == "heading"
#         and (
#             "EXAMPLES" in tag.text.upper()
#             or "EXAMPLE" == tag.text.upper()
#             or "EXPERIMENT" == tag.text.upper()
#             or "EXPERIMENTS" in tag.text.upper()
#         )
#     )
#     if not examples_heading:
#         # print("No 'Examples' section found.")
#         return None


#     return examples_heading 

In [5]:

doc_w_exp = {}
i = 0
for xml in xml_no_dup:
    i += 1
    if i % 100 == 0:
        print(f"{i}/{len(xml_no_dup)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            for example in heading:
                extracted_ex_start_w_word = extract_examples_start_w_word(example.find_next_siblings())
                if len(extracted_ex_start_w_word) > 0:
                    extracted_examples.append(extracted_ex_start_w_word)
                else:
                    extracted_ex_w_word = process_siblings(example.find_next_siblings())
                    if extracted_ex_w_word:
                        if len(extracted_ex_w_word) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    else:
                        num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                        if len(num_dot_examples) > 0:
                            extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = extract_examples_start_w_word(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = process_siblings(heading[0].find_next_siblings())
                    if len(extracted_ex_w_word) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = process_siblings(heading[0].find_next_siblings())
                if extracted_ex_w_word:
                    if len(extracted_ex_w_word) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                    if len(num_dot_examples) > 0:
                        extracted_examples.append(num_dot_examples)
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                extracted_examples.append(extracted_ex_w_word)
            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        
    
        if len(extracted_examples) > 0:
            doc_w_exp[int(find_doc_number(xml)[0])] = extracted_examples
            # if len(doc_w_exp) == 100:
            #     break


"""100/8048 so far found 17 docs with experiments
200/8048 so far found 30 docs with experiments
300/8048 so far found 33 docs with experiments
400/8048 so far found 38 docs with experiments
500/8048 so far found 42 docs with experiments
600/8048 so far found 45 docs with experiments
700/8048 so far found 46 docs with experiments
800/8048 so far found 90 docs with experiments
900/8048 so far found 146 docs with experiments
1000/8048 so far found 187 docs with experiments
1100/8048 so far found 233 docs with experiments
1200/8048 so far found 251 docs with experiments
1300/8048 so far found 258 docs with experiments
1400/8048 so far found 263 docs with experiments
1500/8048 so far found 294 docs with experiments
1600/8048 so far found 298 docs with experiments
1700/8048 so far found 302 docs with experiments
1800/8048 so far found 320 docs with experiments
1900/8048 so far found 326 docs with experiments
2000/8048 so far found 329 docs with experiments
2100/8048 so far found 331 docs with experiments
2200/8048 so far found 332 docs with experiments
2300/8048 so far found 334 docs with experiments
2400/8048 so far found 361 docs with experiments
2500/8048 so far found 435 docs with experiments
with dup: 1196   out of 8048 in 12 minutes"""

"""100/7824 so far found 20 docs with experiments
200/7824 so far found 34 docs with experiments
300/7824 so far found 36 docs with experiments
400/7824 so far found 42 docs with experiments
500/7824 so far found 45 docs with experiments
600/7824 so far found 49 docs with experiments
700/7824 so far found 51 docs with experiments
800/7824 so far found 98 docs with experiments
900/7824 so far found 162 docs with experiments
1000/7824 so far found 217 docs with experiments
1100/7824 so far found 253 docs with experiments
1200/7824 so far found 261 docs with experiments
1300/7824 so far found 266 docs with experiments
1400/7824 so far found 295 docs with experiments
1500/7824 so far found 304 docs with experiments
1600/7824 so far found 309 docs with experiments
1700/7824 so far found 328 docs with experiments
1800/7824 so far found 333 docs with experiments
1900/7824 so far found 339 docs with experiments
2000/7824 so far found 342 docs with experiments
2100/7824 so far found 343 docs with experiments
2200/7824 so far found 345 docs with experiments
2300/7824 so far found 358 docs with experiments
2400/7824 so far found 439 docs with experiments
2500/7824 so far found 510 docs with experiments
no dup 949  out of 7824 in 13.5 minutes"""

100/7824 so far found 20 docs with experiments
200/7824 so far found 34 docs with experiments
300/7824 so far found 36 docs with experiments
400/7824 so far found 42 docs with experiments
500/7824 so far found 45 docs with experiments
600/7824 so far found 49 docs with experiments
700/7824 so far found 51 docs with experiments
800/7824 so far found 98 docs with experiments
900/7824 so far found 162 docs with experiments
1000/7824 so far found 217 docs with experiments
1100/7824 so far found 253 docs with experiments
1200/7824 so far found 261 docs with experiments
1300/7824 so far found 266 docs with experiments
1400/7824 so far found 295 docs with experiments
1500/7824 so far found 304 docs with experiments
1600/7824 so far found 309 docs with experiments
1700/7824 so far found 328 docs with experiments
1800/7824 so far found 333 docs with experiments
1900/7824 so far found 339 docs with experiments
2000/7824 so far found 342 docs with experiments
2100/7824 so far found 343 docs with 

'1196   out of 8048 in 12 minutes'

In [300]:
# save_as_json(doc_w_exp)

Saved as 1200_patents_w_experiments.json


In [11]:
def process_document_examples(xml_parts, max_docs=100):
    """Process and extract examples from XML documents.
    
    Args:
        xml_parts (list): List of XML document parts
        max_docs (int): Maximum number of documents to process
        
    Returns:
        dict: Document numbers mapped to extracted examples
    """
    doc_with_examples = {}
    
    for xml in xml_parts:
        example_headings = extract_experiments_w_heading(xml)
        if not example_headings:
            continue
            
        extracted_examples = []
        
        try:
            if len(example_headings) > 1:
                # Process multiple headings
                for heading in example_headings:
                    siblings = heading.find_next_siblings()
                    examples = extract_examples_start_w_word(siblings)
                    
                    if examples:
                        extracted_examples.append(examples)
                    else:
                        processed = process_siblings(siblings)
                        if processed:
                            extracted_examples.append(processed)
                            
            else:
                # Process single heading
                siblings = example_headings[0].find_next_siblings()
                examples = extract_examples_start_w_word(siblings)
                
                if examples and examples[0]["content"]:
                    extracted_examples.append(examples)
                else:
                    processed = process_siblings(siblings)
                    if processed:
                        extracted_examples.append(processed)
            
            # Fall back to direct XML processing if needed
            if not extracted_examples:
                examples = extract_examples_w_word(xml)
                if examples:
                    extracted_examples.append(examples)
            
            # Store results if examples found
            if extracted_examples:
                doc_number = int(find_doc_number(xml)[0])
                doc_with_examples[doc_number] = extracted_examples
                
                if len(doc_with_examples) >= max_docs:
                    break
                    
        except Exception as e:
            print(f"Error processing document: {str(e)}")
            continue
            
    return doc_with_examples
